In [94]:
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [95]:
train_data=pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')
train_data=train_data.set_index('id')
test_data=pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')
test_data=test_data.set_index('id')
train_data.dropna(inplace=True)
train_data=train_data.drop(['numclaims','claimcst0'],axis=1)
label_mapping_body = {'BUS':0, 'CONVT':1, 'COUPE':2,'HBACK':3,'HDTOP':4,'MCARA':5, 'MIBUS':6,'PANVN':7, 'RDSTR':8, 'SEDAN':9,'STNWG':10, 'TRUCK':11,'UTE':12}
label_mapping_gender={'M':1, 'F':2}
label_mapping_area={'A':1, 'B':2,'C':3,'D':4,'E':5,'F':6}
label_mapping_engine={'petrol':1, 'electric':2, 'dissel':3, 'hybrid':4}
label_mapping_color={'black':1, 'white':2, 'gray':3, 'blue':4, 'brown':5, 'silver':6,'green':7, 'red':8, 'yellow':9}
label_mapping_status={'S':0, 'M':1}
label_mapping_week={'weekday':0,'weekend':1}
label_mapping_time={'6pm - 12am':0, '6am - 12pm':1, '12pm - 6pm':2, '12am - 6 am':3}

train_data['veh_body_label'] = train_data['veh_body'].map(label_mapping_body)
train_data['gender_label'] = train_data['gender'].map(label_mapping_gender)
train_data['area_label'] = train_data['area'].map(label_mapping_area)
train_data['engine_type_label'] = train_data['engine_type'].map(label_mapping_engine)
train_data['veh_color_label'] = train_data['veh_color'].map(label_mapping_color)
train_data['status_label'] = train_data['marital_status'].map(label_mapping_status)
train_data['week_label'] = train_data['time_of_week_driven'].map(label_mapping_week)
train_data['time_label'] = train_data['time_driven'].map(label_mapping_time)

test_data['veh_body_label'] = test_data['veh_body'].map(label_mapping_body)
test_data['gender_label'] = test_data['gender'].map(label_mapping_gender)
test_data['area_label'] = test_data['area'].map(label_mapping_area)
test_data['engine_type_label'] = test_data['engine_type'].map(label_mapping_engine)
test_data['veh_color_label'] = test_data['veh_color'].map(label_mapping_color)
test_data['status_label'] = test_data['marital_status'].map(label_mapping_status)
test_data['week_label'] = test_data['time_of_week_driven'].map(label_mapping_week)
test_data['time_label'] = test_data['time_driven'].map(label_mapping_time)

Drop_column=['veh_body','gender','engine_type','area','veh_color','marital_status','time_of_week_driven','time_driven']
train_data1=train_data.drop(columns=Drop_column,axis=1)
X_train=train_data1.drop('clm',axis=1)
y_train=train_data1['clm']
X_test_vh=test_data.drop(columns=Drop_column,axis=1)



In [96]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [97]:
resampling_pipeline = Pipeline([
    ('oversample', SMOTE(sampling_strategy=0.5)),  # Adjust the sampling_strategy as needed
    ('undersample', RandomUnderSampler(sampling_strategy=0.8))  # Adjust the sampling_strategy as needed
])


In [98]:
X_resampled, y_resampled = resampling_pipeline.fit_resample(X_train, y_train)

In [99]:
classifier = RandomForestClassifier()
classifier.fit(X_resampled, y_resampled)

RandomForestClassifier()

In [100]:
y_pred = classifier.predict(X_test)
X_test.loc[:, 'clm'] = y_pred
X_test.to_csv('result_test.csv', index=False)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8908045977011494
Confusion Matrix:
 [[4009  198]
 [ 296   21]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      4207
           1       0.10      0.07      0.08       317

    accuracy                           0.89      4524
   macro avg       0.51      0.51      0.51      4524
weighted avg       0.87      0.89      0.88      4524



In [101]:
y_pred_vh = classifier.predict(X_test_vh)
X_test_vh.loc[:, 'clm'] = y_pred_vh
X_test_vh.to_csv('result_vh.csv', index=False)

In [102]:
X_test_vh.clm.value_counts()

clm
0    21426
1     1194
Name: count, dtype: int64